In [1]:
import pandas as pd
import numpy as np

In [18]:
from sentence_transformers import SentenceTransformer

In [31]:
import faiss

## Tasks

1- Reads and preprocesses customer service conversations from a CSV file.

2- Generates embeddings for customer messages using sentence-transformers.

3- Stores embeddings in a vector database (FAISS) for fast similarity search.

4- Implements a function to retrieve the top 3 most relevant responses based on a user query.


In [45]:
dataset = pd.read_csv('./customer_service_conversations.csv')
dataset

,ID,Customer Message,Agent Response
0,1,My order hasn’t arrived yet. Can you check?,I’m sorry for the delay! Can you provide your ...
1,2,I need to reset my password but the link is br...,I understand. Let me generate a new reset link...
2,3,Do you have a refund policy?,Yes! We offer a 30-day money-back guarantee. W...
3,4,The product I received is defective.,I'm sorry to hear that. We can arrange a repla...
4,5,How can I contact support?,You can reach us via email at support@example....


In [46]:
## preprocessing data
dataset['Customer Message'] = dataset['Customer Message'].apply(lambda x: x.lower())
dataset['Agent Response'] = dataset['Agent Response'].apply(lambda x: x.lower())

In [47]:
dataset

,ID,Customer Message,Agent Response
0,1,my order hasn’t arrived yet. can you check?,i’m sorry for the delay! can you provide your ...
1,2,i need to reset my password but the link is br...,i understand. let me generate a new reset link...
2,3,do you have a refund policy?,yes! we offer a 30-day money-back guarantee. w...
3,4,the product i received is defective.,i'm sorry to hear that. we can arrange a repla...
4,5,how can i contact support?,you can reach us via email at support@example....


In [49]:
responses = dataset['Agent Response'].tolist()
responses

['i’m sorry for the delay! can you provide your order number?',
 'i understand. let me generate a new reset link for you.',
 'yes! we offer a 30-day money-back guarantee. would you like me to process a refund?',
 "i'm sorry to hear that. we can arrange a replacement or refund. what would you prefer?",
 'you can reach us via email at support@example.com or call our helpline.']

In [50]:
## applying sentence transformers
model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(responses)


In [51]:
## vector database
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)  # Using L2 (Euclidean) distance
index.add(embeddings)


In [53]:
## top 3 most relevant answers
user_query =  dataset['Customer Message'].iloc[0]
query_embedding = model.encode([user_query])

k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Display the results
print(f"Query: {user_query}")

print("Most similar sentences:")
for i, idx in enumerate(indices[0]):
    print(f"{i + 1}: {responses[idx]} (Distance: {distances[0][i]})")

Query: my order hasn’t arrived yet. can you check?
Most similar sentences:
1: i’m sorry for the delay! can you provide your order number? (Distance: 0.6857123970985413)
2: yes! we offer a 30-day money-back guarantee. would you like me to process a refund? (Distance: 1.4049072265625)
3: i'm sorry to hear that. we can arrange a replacement or refund. what would you prefer? (Distance: 1.5908550024032593)
